In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense


In [ ]:
# Load data
data = pd.read_csv("cold_season_NITJ_data.csv", parse_dates=['Date'])

# Feature selection
X = data[['Temperature', 'Pressure', 'Relative Humidity', 'Solar Zenith Angle', 'Wind Speed', 'Wind Direction', 'Dew Point', 'Precipitable Water', 'Clearness Index']]
y = data['GHI']


In [ ]:
# Data normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data for CNN
X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)


In [ ]:
# Define function to create CNN model
def create_model():
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from keras.models import Sequential
from keras.layers import Dense

class KerasCustomRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, batch_size=32, epochs=50, optimizer='adam'):
        self.batch_size = batch_size
        self.epochs = epochs
        self.optimizer = optimizer
        self.model = None

    def fit(self, X, y):
        self.model = Sequential()
        self.model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
        self.model.add(Dense(1, activation='linear'))
        self.model.compile(loss='mean_squared_error', optimizer=self.optimizer)
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self

    def predict(self, X):
        return self.model.predict(X).ravel()


Cross validations

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression

# Generate some example data
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1)

# Define hyperparameters for grid search
param_grid = {
    'batch_size': [16, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'sgd', 'rmsprop']
}

# Perform grid search with cross-validation
model = KerasCustomRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_result = grid_search.fit(X, y)

# Print best parameters and score
print("Best Parameters:", grid_result.best_params_)
print("Best Score:", grid_result.best_score_)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
11/11 [==============================] - 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py", line 442, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py", line 102, in _check_reg_targets
    y_pred = check_array(

11/11 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py", line 442, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py", line 102, in _check_reg_targets
    y_pred = check_array(

11/11 [==============================] - 0s 2ms/step


...

11/11 [==============================] - 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py", line 442, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py", line 102, in _check_reg_targets
    y_pred = check_array(

11/11 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [ -525.15689584            nan  -571.72525917  -249.66096446
            nan   -95.42915983 -1771.83244206            nan
 -6039.42932563  -466.78554484            nan  -371.21023064]
  warnings.warn(


Best Parameters: {'batch_size': 16, 'epochs': 100, 'optimizer': 'rmsprop'}
Best Score: -95.4291598303738


In [ ]:

# Perform grid search with cross-validation on the training set

# Evaluate the best model on test set
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)


r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("R2 Score:", r2)
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)



7/7 [==============================] - 0s 2ms/step
R2 Score: 0.9999809027507387
RMSE: 0.7562842635544819
MSE: 0.571965887300145
MAE: 0.5541902120544472
